In [1]:
# import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

# Loading required packages
import spacy
import pandas as pd
import re
# import gensim
# import gensim.downloader
# from gensim.models import Word2Vec
# from gensim.test.utils import common_texts
# from gensim.models import Word2Vec
# from gensim.models.phrases import Phrases, Phraser
import nltk
from numpy import mean
from numpy import std
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt  
from sklearn.metrics import plot_confusion_matrix
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from statistics import mean
import json
import csv
import ast
import numpy as np

# from flair.embeddings import WordEmbeddings
# from flair.data import Sentence
# from flair.embeddings import TransformerWordEmbeddings

from tqdm import tqdm
import pickle 
# Set seed
seed = np.random.seed(1)


# Select Spacy model
# Efficiency
nlp = spacy.load("en_core_web_sm")

# Accuracy
# nlp = spacy.load("en_core_web_trf")

# Change working directory
# %cd '/Users/richardfrissen/Documents/Maastricht University/Thesis/Development/Datasets'

In [2]:
class_names=['Demographic and Racial language', 'Exclusive language', 'Feminine-coded words', 'LGBTQ-coloured language', 'Masculine-coded words', 'O']

In [3]:
%%time

# Optionally the data can be saved to create a checkpoint

# data.to_csv('data_features_full_wordembedding_glove.csv', index = False)

data = pd.read_csv('data_features_full_wordembedding_glove.csv', nrows = 1000, na_values=['nan'])

data.head(10)

CPU times: user 5.99 ms, sys: 4.06 ms, total: 10 ms
Wall time: 9.52 ms


,Token,Label,pos,ent_type,is_alpha,is_ascii,is_digit,is_lower,is_upper,is_title,is_punct,is_space,like_num,is_oov,is_stop,lang,sentiment,word_length,0
0,food,O,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.164860
1,a,O,90,0,True,True,False,True,False,False,False,False,False,True,True,14626626061804382878,0.0,1,-0.270860
2,fast,O,86,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.294280
3,grow,O,100,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.128200
4,-winne,O,97,0,False,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,6,-0.130350
5,online,O,86,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,6,0.124870
6,food,O,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.164860
7,community,O,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,9,0.370750
8,and,O,89,0,True,True,False,True,False,False,False,False,False,True,True,14626626061804382878,0.0,3,-0.071953
9,crowd,O,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,5,0.325750


In [4]:
data["Label"] = data["Label"].astype('category').cat.codes

In [5]:
%%time

# Replace NaN values with a "0"

data = data.replace(np.nan, "0", regex=True)

# We drop the token, as it is no longer needed for prediction
data.drop('Token', axis=1, inplace=True)
data.head(10)

# data = data.stack().str.decode('utf-8').unstack()

CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 2.45 ms


,Label,pos,ent_type,is_alpha,is_ascii,is_digit,is_lower,is_upper,is_title,is_punct,is_space,like_num,is_oov,is_stop,lang,sentiment,word_length,0
0,2,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.164860
1,2,90,0,True,True,False,True,False,False,False,False,False,True,True,14626626061804382878,0.0,1,-0.270860
2,2,86,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.294280
3,2,100,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.128200
4,2,97,0,False,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,6,-0.130350
5,2,86,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,6,0.124870
6,2,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,4,-0.164860
7,2,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,9,0.370750
8,2,89,0,True,True,False,True,False,False,False,False,False,True,True,14626626061804382878,0.0,3,-0.071953
9,2,92,0,True,True,False,True,False,False,False,False,False,True,False,14626626061804382878,0.0,5,0.325750


In [6]:
%%time

# 80% / 20% split
# Train, Test = train_test_split(data1, test_size=0.2, shuffle=False)

X = data.drop(['Label'],axis=1).values # independant features
y = data['Label'].values # dependant variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

CPU times: user 3.31 ms, sys: 0 ns, total: 3.31 ms
Wall time: 3.31 ms


In [7]:
max_iterations = 1000000000

In [8]:
# Optionally the data can be saved to create a checkpoint - RF

import pickle

# f = open('bert_rf.pckl', 'wb')
# pickle.dump(clf, f)
# f.close()

f = open('glove_svm_prob.pckl', 'rb')
clf = pickle.load(f)
f.close()

In [9]:
type(X)

numpy.ndarray

In [10]:
X_featurenames = (pd.DataFrame(X_train)).columns

In [11]:
len(X_featurenames)

17

In [12]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)

In [13]:
# %time
# explainer = shap.TreeExplainer(clf)
# shap_values = explainer.shap_values(X_train, check_additivity=False)

In [14]:
import shap

In [15]:
X_train = shap.utils.sample(X_train, nsamples=100, random_state=0)

In [ ]:
%time
explainer = shap.KernelExplainer(clf.predict_proba, X_train)
shap_values = explainer.shap_values(X_test)

CPU times: user 2 µs, sys: 9 µs, total: 11 µs
Wall time: 21.9 µs


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
%time
shap.summary_plot(shap_values[0], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[1], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[2], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[3], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[4], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[5], X_train, plot_type="bar")

In [ ]:
%time
shap.summary_plot(shap_values[0], X_train, feature_names = X_train.columns)

In [ ]:
%time
shap.summary_plot(shap_values[1], X_train, feature_names = X_train.columns)

In [ ]:
%time
shap.summary_plot(shap_values[2], X_train, feature_names = X_train.columns)

In [ ]:
%time
shap.summary_plot(shap_values[3], X_train, feature_names = X_train.columns)

In [ ]:
%time
shap.summary_plot(shap_values[4], X_train, feature_names = X_train.columns)

In [ ]:
%time
shap.summary_plot(shap_values[5], X_train, feature_names = X_train.columns)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# Optionally the data can be saved to create a checkpoint - Shap RF

import pickle

f = open('shap_values_glove_svm.pckl', 'wb')
pickle.dump(shap_values, f)
f.close()

# f = open('shap_values_word2vec_svm.pckl', 'rb')
# shap_values = pickle.load(f)
# f.close()